In [23]:
%load_ext autoreload
%autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import os
import shutil
import re
import pickle
from src.data_loader import extract_latex_dependencies, build_resume_context, flatten_pydantic
from src.data_loader import rebase_dependency_paths, convert_to_latex, create_dep_map, write_section_content
from src.llm_tools import enrich_file_metadata, extract_jd_features, tailor_profile_and_highlights


In [25]:
# check for the cache file containing all the dependencies and load it if it exists
cache_dir = "./cache"
# check the validity of the cache directory
if not os.path.exists(cache_dir):
    raise FileNotFoundError(f"The directory was not found: ' {cache_dir} ' ")

cache_file_name = "resume_metadata.pkl"
cache_path = os.path.join(cache_dir, cache_file_name)

if os.path.exists(cache_path): 
    with open(cache_path, 'rb') as f: 
        dep_list = pickle.load(f)
        print("File loaded")
#else proccess the latex main file for the dependencies list
else: 
    rsm_main_path = "./Rsm/main/Resume_Bardia_Azami/resume-general/Bardia-Azami-Resume.tex"
    #check the validity of file path 
    if not os.path.exists(rsm_main_path): 
        raise FileNotFoundError(f"The directory was not found: ' {rsm_main_path} ' ")
    #unproccesed dependencies list
    dep_unprc = extract_latex_dependencies(rsm_main_path)
    #enrich the dependencies list 
    dep_list = enrich_file_metadata(dep_unprc)
    print("dependencies extracted")
    #save the list 
    with open(cache_path, 'wb') as f: 
        pickle.dump(dep_list, f)
        print("The proccesed dependencies saved")


File loaded


In [26]:
# Create a mpping function from the llm output keys to the extracted dependencies keys 
LLM_TO_RESUME_BRIDGE = {
    # -- Summaries --
    "profile": ("PROFILE", "Professional Summary"),
    "highlights": ("HIGHLIGHT OF QUALIFICATIONS", "Qualifications Summary"), 
    # -- Experience / Projects --
    "housing_project": ("TECHNICAL EXPERIENCE", "Machine Learning Engineer - Ottawa Housing Demand Analysis"),
    "nlp_project": ("TECHNICAL EXPERIENCE", "AI Agent Developer – RAG-Based QA System (WW2 Dataset)"),
    "ros2_project": ("TECHNICAL EXPERIENCE", "Robotics Engineer - Algonquin College"),
    "behyar_job": ("TECHNICAL EXPERIENCE", "Computer Vision Engineer - Behyar Sanaat Sepahan"),
    "ui_research": ("TECHNICAL EXPERIENCE", "AI Researcher - Univ of Isfahan"),
    # -- Skills --
    "skills": ("TECHNICAL SKILLS", "Technical Skills")
}

In [27]:
# Define the directories
jd_dir = "./Rsm/jd"
rsm_dir = "./Rsm/main/Resume_Bardia_Azami"
rsm_tailored_path ="./Rsm/tailored"
for path in [jd_dir, rsm_dir, rsm_tailored_path]:
    if not os.path.exists(path):
        raise FileExistsError(f"could not find the job description file:{jd_dir} ")

# list all .tex files in jd directory
tex_files = [f for f in os.listdir(jd_dir) if f.endswith(".txt")]

# create destination folders for tailored resumes and copy the main resume for making adjusments
for tex_file in tex_files: 
    target_dir = os.path.join( rsm_tailored_path , os.path.splitext(tex_file)[0] )
    os.makedirs(target_dir, exist_ok=True)

    destination = os.path.join(target_dir, os.path.basename(rsm_dir))
    if not os.path.exists(destination):
        shutil.copytree(rsm_dir, destination)

    #update dep list for the tailored resume path 
    new_dep_list = rebase_dependency_paths(dep_list , destination, rsm_dir)
    dep_map = create_dep_map(new_dep_list)



Rebasing paths...
FROM: ./Rsm/main/Resume_Bardia_Azami
TO:   ./Rsm/tailored/GPTZero(Machine Learning Intern)/Resume_Bardia_Azami



In [28]:
#TEMP
jd_path = "./Rsm/jd/GPTZero(Machine Learning Intern).txt"

if not os.path.exists(jd_path):
    raise FileNotFoundError(f"File not found: {jd_file_path}")
        
try:
    with open(jd_path, 'r', encoding='utf-8') as f:
        jd = f.read()
except Exception as e:
    raise IOError(f"Error reading file: {e}")
    
jd_kw = extract_jd_features(jd)

#TEMP

Step 1: Mining keywords (High Recall)...
Minerd found 40 potential keywords.
Step 2: Judging keywords (High Precision)...
Judge has finished filtering.


In [30]:
jd_kw_ls = flatten_pydantic(jd_kw , ['domain_knowledge', 'technical_stack', 'tools_and_platforms', 'soft_skills'])
print(len(jd_kw_ls))

12


In [31]:
rsm = build_resume_context(dep_list) 

In [32]:
data = tailor_profile_and_highlights(rsm, jd, jd_kw_ls)

In [42]:
for keys in data.keys():
    latex_data = convert_to_latex( data[keys] )
    section, title = LLM_TO_RESUME_BRIDGE[keys]
    write_section_content(dep_map, section , title , latex_data)

Updated: Professional Summary
Updated: Qualifications Summary
